## Description:
This model will be trained on the images classified as 'Male' by the Trousers_and_Jeans_Gender_Split model, and then further classify these images into the different types of trousers and jeans.

The targets for the dataset are 'Trousers' and 'Jeans', and this makes this task a binary classification problem.  
However, because of the hierarchical order of the models, these targets are really 'Male Trousers' and 'Male Jeans'.

In [1]:
import io
import itertools

import numpy as np
import sklearn.metrics

import tensorflow as tf
from tensorboard.plugins.hparams import api as hp

import matplotlib.pyplot as plt

from os import environ # To get the file path of the datasets location from Windows environment variables.

## Downloading and preprocessing the data

In [2]:
# load the datasets from the path stored in the operating system's environment variable
dataset_path = environ.get('CNN_Practical_Dataset_Path')

data_train = np.load(dataset_path + "/Trousers & Jeans - Male - Train.npz")
data_validation = np.load(dataset_path + "/Trousers & Jeans - Male - Validation.npz")
data_test = np.load(dataset_path + "/Trousers & Jeans - Male - Test.npz")

In [3]:
# Extracting the arrays from the imported data
images_train = data_train['images']
labels_train = data_train['labels']

images_val = data_validation['images']
labels_val = data_validation['labels']

images_test = data_test['images']
labels_test = data_test['labels']

In [4]:
# Scaling the pixel values of all images
images_train = images_train/255.0
images_val = images_val/255.0
images_test = images_test/255.0

In [5]:
# Defining constants
EPOCHS = 20
BATCH_SIZE = 64

In [6]:
# Defining the hyperparameters we would tune, and their values to be tested
HP_FILTER_SIZE = hp.HParam('filter_size', hp.Discrete([5,7]))
HP_FILTER_NUM = hp.HParam('filters_number', hp.Discrete([64,96]))
HP_DENSE_SIZE = hp.HParam('dense_size', hp.Discrete([256,512]))

METRIC_ACCURACY = 'accuracy'

# Logging setup info
with tf.summary.create_file_writer(r'Logs_Practical/Model Trousers_and_Jeans_Type_Male/hparam_tuning/').as_default():
    hp.hparams_config(
        hparams=[HP_FILTER_SIZE, HP_FILTER_NUM, HP_DENSE_SIZE],
        metrics=[hp.Metric(METRIC_ACCURACY, display_name='Accuracy')],
    )

## Creating the model and training it

Now that we have preprocessed the dataset, we can define our CNN and train it

In [7]:
# Wrapping our model and training in a function
def train_test_model(hparams, session_num):
    
    # Outlining the model/architecture of our CNN
    model = tf.keras.Sequential([
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu', input_shape=(120,90,3)),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Conv2D(hparams[HP_FILTER_NUM], hparams[HP_FILTER_SIZE], activation='relu'),
        tf.keras.layers.MaxPooling2D(pool_size=(2,2)),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(hparams[HP_DENSE_SIZE], activation='relu'),
        tf.keras.layers.Dense(2, activation='softmax')
    ])
    
    # Defining the loss function
    loss_fn = tf.keras.losses.BinaryCrossentropy(from_logits=False)

    # Compiling the model
    model.compile(optimizer='adam', loss=loss_fn, metrics=['accuracy'])
    
    # Defining the logging directory
    log_dir = "Logs_Practical\\Model Trousers_and_Jeans_Type_Male\\fit\\" + "run-{}".format(session_num)
    
    def plot_confusion_matrix(cm, class_names):
        """
        Returns a matplotlib figure containing the plotted confusion matrix.

        Args:
          cm (array, shape = [n, n]): a confusion matrix of integer classes
          class_names (array, shape = [n]): String names of the integer classes
        """
        
        figure = plt.figure(figsize=(12, 12))
        plt.imshow(cm, interpolation='nearest', cmap=plt.cm.Blues)
        plt.title("Confusion matrix")
        plt.colorbar()
        tick_marks = np.arange(len(class_names))
        plt.xticks(tick_marks, class_names, rotation=45)
        plt.yticks(tick_marks, class_names)

        # Normalize the confusion matrix.
        cm = np.around(cm.astype('float') / cm.sum(axis=1)[:, np.newaxis], decimals=2)

        # Use white text if squares are dark; otherwise black.
        threshold = cm.max() / 2.
        for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
            color = "white" if cm[i, j] > threshold else "black"
            plt.text(j, i, cm[i, j], horizontalalignment="center", color=color)

        plt.tight_layout()
        plt.ylabel('True label')
        plt.xlabel('Predicted label')
        return figure
    
    
    def plot_to_image(figure):
        """Converts the matplotlib plot specified by 'figure' to a PNG image and
        returns it. The supplied figure is closed and inaccessible after this call."""
        # Save the plot to a PNG in memory.
        buf = io.BytesIO()
        plt.savefig(buf, format='png')
        # Closing the figure prevents it from being displayed directly inside
        # the notebook.
        plt.close(figure)
        buf.seek(0)
        # Convert PNG buffer to TF image
        image = tf.image.decode_png(buf.getvalue(), channels=4)
        # Add the batch dimension
        image = tf.expand_dims(image, 0)
        return image
    
    
    # Defining a file writer for Confusion Matrix logging purposes
    file_writer_cm = tf.summary.create_file_writer(log_dir + '/cm')
    
    
    def log_confusion_matrix(epoch, logs):
        # Use the model to predict the values from the validation dataset.
        test_pred_raw = model.predict(images_val)
        test_pred = np.argmax(test_pred_raw, axis=1)

        # Calculate the confusion matrix.
        cm = sklearn.metrics.confusion_matrix(labels_val, test_pred)
        # Log the confusion matrix as an image summary.
        figure = plot_confusion_matrix(cm, class_names=['Trousers', 'Jeans'])
        cm_image = plot_to_image(figure)

        # Log the confusion matrix as an image summary.
        with file_writer_cm.as_default():
            tf.summary.image("Confusion Matrix", cm_image, step=epoch)
    
    
    # Define the Tensorboard and Confusion Matrix callbacks.
    tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1, profile_batch=0)
    cm_callback = tf.keras.callbacks.LambdaCallback(on_epoch_end=log_confusion_matrix)

    
    # Defining early stopping to prevent overfitting
    early_stopping = tf.keras.callbacks.EarlyStopping(
        monitor = 'val_loss',
        mode = 'auto',
        min_delta = 0,
        patience = 2,
        verbose = 0, 
        restore_best_weights = True
    )
    
    # Training the model
    model.fit(
        images_train,
        labels_train,
        epochs = EPOCHS,
        batch_size = BATCH_SIZE,
        callbacks = [tensorboard_callback, cm_callback, early_stopping],
        validation_data = (images_val,labels_val),
        verbose = 2
    )
    
    
    # Evaluating the model's performance on the validation set
    _, accuracy = model.evaluate(images_val,labels_val)
    
    # Saving the current model for future reference
    model.save(r"saved_models\Model Trousers_and_Jeans_Type_Male\Run-{}".format(session_num))
    
    return accuracy

In [8]:
# Creating a function to log the results
def run(log_dir, hparams, session_num):
    
    with tf.summary.create_file_writer(log_dir).as_default():
        hp.hparams(hparams)  # record the values used in this trial
        accuracy = train_test_model(hparams, session_num)
        tf.summary.scalar(METRIC_ACCURACY, accuracy, step=1)

In [9]:
session_num = 1

for filter_size in HP_FILTER_SIZE.domain.values:
    for filter_num in HP_FILTER_NUM.domain.values:
        for dense_size in HP_DENSE_SIZE.domain.values:

            hparams = {
                HP_FILTER_SIZE: filter_size,
                HP_FILTER_NUM: filter_num,
                HP_DENSE_SIZE: dense_size
            }

            run_name = "run-%d" % session_num
            print('--- Starting trial: %s' % run_name)
            print({h.name: hparams[h] for h in hparams})
            run('Logs_Practical/Model Trousers_and_Jeans_Type_Male/hparam_tuning/' + run_name, hparams, session_num)

            session_num += 1

--- Starting trial: run-1
{'filter_size': 5, 'filters_number': 64, 'dense_size': 256}
Epoch 1/20
32/32 - 35s - loss: 7.4457 - accuracy: 0.4995 - val_loss: 7.6661 - val_accuracy: 0.4720
Epoch 2/20
32/32 - 33s - loss: 7.6687 - accuracy: 0.5020 - val_loss: 7.6661 - val_accuracy: 0.4720
Epoch 3/20
32/32 - 52s - loss: 7.6687 - accuracy: 0.5020 - val_loss: 7.6661 - val_accuracy: 0.4720
8/8 [==============================] - 1s 185ms/step - loss: 7.6661 - accuracy: 0.4720
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
Instructions for updating:
This property should not be used in TensorFlow 2.0, as updates are applied automatically.
INFO:tensorflow:Assets written to: saved_models\Model Trousers_and_Jeans_Type_Male\Run-1\assets
--- Starting trial: run-2
{'filter_size': 5, 'filters_number': 64, 'dense_size': 512}
Epoch 1/20
32/32 - 38s - loss: 7.4456 - accuracy: 0.4960 - val_loss: 7.6661 - val_accuracy: 0.4720
Epoch 2/20
32/3

In [10]:
# Loading a model to evaluate on the test set
model = tf.keras.models.load_model(r"saved_models\Model Trousers_and_Jeans_Type_Male\Run-1")

In [11]:
test_loss, test_accuracy = model.evaluate(images_test,labels_test)

8/8 [==============================] - 1s 126ms/step - loss: 7.6696 - accuracy: 0.0000e+00


In [12]:
# Printing the test results
print('Test loss: {0:.4f}. Test accuracy: {1:.2f}%'.format(test_loss, test_accuracy*100.))

Test loss: 7.6696. Test accuracy: 0.00%


## Visualizing in Tensorboard

In [15]:
%load_ext tensorboard
%tensorboard --logdir "Logs_Practical/Model Trousers_and_Jeans_Type_Male/hparam_tuning"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


Reusing TensorBoard on port 6006 (pid 12612), started 2:55:33 ago. (Use '!kill 12612' to kill it.)

In [14]:
%load_ext tensorboard
%tensorboard --logdir "Logs_Practical/Model Trousers_and_Jeans_Type_Male/fit"

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


ERROR: Timed out waiting for TensorBoard to start. It may still be running as pid 16480.